In [ ]:
import gzip
import json
import os
import pprint
import subprocess
import time
from datetime import timedelta
from pathlib import Path

import numpy as np
import pandas as pd
import requests

In [ ]:
weaviate_name = "benchmark_weaviate"
weaviate_host = "localhost"
weaviate_port = 8091
weaviate_version = "1.24.6"

In [ ]:
def get_dataset_config(target_name):
    setting = {
        "100k-768-m49-ef100-ip": {
            "content_path": "dataset/passages-c400-jawiki-20230403",
            "embedding_path": "dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage",
            "num_of_docs": 5555583,
            "index_size": 100000,
            "bulk_size": 1000,
            "index_name": "Content",
            "distance": "dot",
            "dimension": 768,
            "hnsw_m": 48,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
        },
        "1m-768-m49-ef100-ip": {
            "content_path": "dataset/passages-c400-jawiki-20230403",
            "embedding_path": "dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage",
            "num_of_docs": 5555583,
            "index_size": 1000000,
            "bulk_size": 1000,
            "index_name": "Content",
            "distance": "dot",
            "dimension": 768,
            "hnsw_m": 48,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
        },
        "5m-768-m49-ef100-ip": {
            "content_path": "dataset/passages-c400-jawiki-20230403",
            "embedding_path": "dataset/passages-c400-jawiki-20230403/multilingual-e5-base-passage",
            "num_of_docs": 5555583,
            "index_size": 5000000,
            "bulk_size": 1000,
            "index_name": "Content",
            "distance": "dot",
            "dimension": 768,
            "hnsw_m": 48,
            "hnsw_ef_construction": 200,
            "hnsw_ef": 100,
        },
    }
    return setting.get(target_name)

volume_dir = os.getenv("VOLUME_DIR", "./data")

dataset_config = get_dataset_config(os.getenv("TARGET_CONFIG", "100k-768-m49-ef100-ip"))
pprint.pprint(dataset_config)

content_path = Path(dataset_config.get("content_path"))
embedding_path = Path(dataset_config.get("embedding_path"))
num_of_docs = int(dataset_config.get("num_of_docs"))
index_size = int(dataset_config.get("index_size"))
bulk_size = int(dataset_config.get("bulk_size"))

index_name = dataset_config.get("index_name")
distance = dataset_config.get("distance")
dimension = int(dataset_config.get("dimension"))
hnsw_m = int(dataset_config.get("hnsw_m"))
hnsw_ef_construction = int(dataset_config.get("hnsw_ef_construction"))
hnsw_ef = int(dataset_config.get("hnsw_ef"))

results = {}

In [ ]:
def run_weaviate():
    print(f"Starting {weaviate_name}... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "run", "-d",
        "--name", weaviate_name,
        "-p", f"{weaviate_port}:8080",
        # "-v", f"{volume_dir}:/data/content",
        f"cr.weaviate.io/semitechnologies/weaviate:{weaviate_version}"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def stop_weaviate():
    print(f"Stopping {weaviate_name}... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "stop", weaviate_name
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def prune_docker():
    print(f"Cleaning up... ", end="")
    docker_cmd = [
        # "sudo",
        "docker", "system", "prune", "-f"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("[OK]")
    else:
        print(f"[FAIL]")
        print("STDOUT:")
        print(result.stdout)
        print("STDERR:")
        print(result.stderr)


In [ ]:
def print_docker_system_df():
    docker_cmd = [
        # "sudo",
        "docker", "system", "df"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print(result.stdout)
    else:
        print(result.stderr)


In [ ]:
def print_docker_container_stats():
    docker_cmd = [
        # "sudo",
        "docker", "container", "stats", "--no-stream"
    ]
    result = subprocess.run(docker_cmd, capture_output=True, text=True)
    containers = {}
    if result.returncode == 0:
        print(result.stdout)
        for line in result.stdout.split("\n"):
            if line.startswith("CONTAINER") or len(line) == 0:
                continue
            values = line.split()
            containers[values[1]] = {
                "container_id": values[0],
                "cpu": values[2],
                "mem": values[6],
                "mem_usage": values[3],
                "mem_limit": values[5],
                "net_in": values[7],
                "net_out": values[9],
                "block_in": values[10],
                "block_out": values[12],
                "pids": values[13],
            }
    else:
        print(result.stderr)
    return containers


In [ ]:
def create_index():
    print(F"Creating {index_name}... ", end="")
    response = requests.post(f"http://{weaviate_host}:{weaviate_port}/v1/schema",
                            headers={"Content-Type": "application/json"},
                            json={
                                "class": index_name,
                                "vectorIndexType": "hnsw",
                                "vectorIndexConfig": {
                                    "distance": distance,
                                    "maxConnections": hnsw_m,
                                    "ef": hnsw_ef,
                                    "efConstruction": hnsw_ef_construction,
                                },
                                "properties": [
                                    {
                                        "name": "doc_id",
                                        "dataType": ["int"]
                                    },
                                    {
                                        "name": "pageId",
                                        "dataType": ["int"]
                                    },
                                    {
                                        "name": "revId",
                                        "dataType": ["int"]
                                    },
                                    {
                                        "name": "section",
                                        "dataType": ["string"],
                                        "indexInverted": True
                                    },
                                    {
                                        "name": "text",
                                        "dataType": ["text"],
                                        "indexInverted": True
                                    },
                                    {
                                        "name": "title",
                                        "dataType": ["text"],
                                        "indexInverted": True
                                    }
                                ]
                            })
    if response.status_code == 200:
        print("[OK]")
    else:
        print(f"[FAIL]\n{response.text}")


In [ ]:
def delete_index():
    print(F"Deleting {index_name}... ", end="")
    response = requests.delete(f"http://{weaviate_host}:{weaviate_port}/v1/schema/{index_name}")
    if response.status_code == 200:
        print("[OK]")
    else:
        print(f"[FAIL]\n{response.text}")


In [ ]:
def print_indices():
    response = requests.post(f"http://{weaviate_host}:{weaviate_port}/v1/graphql",
                            headers={"Content-Type": "application/json"},
                            json={
                                "query": "{ Aggregate { Content { meta { count } } } }"
                            })
    if response.status_code == 200:
        obj = json.loads(response.text)
        count = obj.get("data").get("Aggregate").get(index_name)[0].get("meta").get("count")
        print(f"count: {count}")
        return {
            "num_of_docs": count,
        }
    print("count: FAILED")
    return {}


In [ ]:
def wait_for_weaviate(retry_count=60):
    print(f"Waiting for {weaviate_name}", end="")
    for i in range(retry_count):
        try:
            response = requests.get(f"http://{weaviate_host}:{weaviate_port}/v1/nodes")
            if response.status_code == 200:
                obj = json.loads(response.text)
                if obj.get("nodes")[0].get("status") == "HEALTHY":
                    print(" [OK]")        
                    return
        except:
            pass
        print(".", end="")
        time.sleep(1)
    print(" [FAIL]")


In [ ]:
def get_embedding(embedding_index, embedding_data, id):
    emb_index = int(id / 100000) * 100000
    if embedding_data is None or embedding_index != emb_index:
        with np.load(embedding_path / f"{emb_index}.npz") as data:
            embedding_data = data["embs"]
    embedding = embedding_data[id - emb_index]
    if distance == "dot":
        embedding = embedding / np.linalg.norm(embedding)
    return emb_index, embedding_data, embedding


section_values = []

def get_section_values(df, min_count=1000):
    tmp_df = df[["id","section"]].groupby("section").count().reset_index()
    tmp_df = tmp_df[tmp_df["id"] >= min_count]
    return tmp_df["section"].values.tolist()


def insert_data(bulk_size, max_size):
    start_time = time.time()

    docs = []
    def send_data(pos):
        print(F"Sending {int(len(docs))} docs ({pos}/{max_size})... ", end="")
        now = time.time()
        response = requests.post(f"http://{weaviate_host}:{weaviate_port}/v1/batch/objects",
                                 headers={"Content-Type": "application/json"},
                                 json={
                                     "objects": docs,
                                 })
        if response.status_code == 200:
            t = time.time() - now
            print(f"[OK] {t}")
            return t
        else:
            print(f"[FAIL] 0 {response.status_code} {response.text}")
            return 0

    total_time = 0
    count = 0
    embedding_index = -1
    embedding_data = None
    for content_file in sorted(content_path.glob("*.parquet")):
        df = pd.read_parquet(content_file)
        section_values.extend(get_section_values(df))
        for i,row in df.iterrows():
            if count >= max_size:
                break
            embedding_index, embedding_data, embedding = get_embedding(embedding_index, embedding_data, row.id)
            count += 1
            docs.append({
                "class": index_name,
                "properties": {
                    "doc_id": count,
                    "page_id": row.pageid,
                    "rev_id": row.revid,
                    #"title": row.title,
                    "section": row.section,
                    #"text": row.text,
                },
                "vector": embedding.tolist(),                    
            })
            if len(docs) >= bulk_size:
                total_time += send_data(count)
                docs = []

    if len(docs) > 0:
        total_time += send_data(count)

    execution_time = time.time() - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Execution Time: {int(hours):02d}:{int(minutes):02d}:{seconds:02.2f} ({timedelta(seconds=total_time)})")
    return {
        "execution_time": execution_time,
        "process_time": total_time,
    }


In [ ]:
def search(query):
    # print(query)
    now = time.time()
    response = requests.post(f"http://{weaviate_host}:{weaviate_port}/v1/graphql",
                             json={
                                 "query": query
                             })
    took = time.time() - now
    # print(response.text)

    if response.status_code == 200:
        obj = json.loads(response.text)
        results = obj.get("data").get("Get").get(index_name)
        product_ids = [x.get("doc_id") for x in results]
        scores = [x.get("_additional").get("distance") for x in results]
        return took * 1000, len(results), product_ids, scores
    print(f"[FAIL][{response.status_code}] {response.text}")
    return -1, -1, [], []


In [ ]:
def create_query(embedding, page_size, pre_filter=None):
    where = next(pre_filter) if pre_filter is not None else ""
    return  f"""{{
  Get {{
    {index_name} (
      limit: {page_size}
      nearVector: {{
        vector: {json.dumps(embedding)}
      }}
{where}
    ) {{
      doc_id
      section
      _additional {{
        distance
      }}
    }}
  }}
}}"""


def search_with_knn_queries(output_path, pre_filter=None, max_size=10000, page_size=100, offset=0):
    print("Sending knn queries...")
    start_time = time.time()
    pos = offset
    count = 0
    running = True
    error_count = 0
    with gzip.open(output_path, "wt", encoding="utf-8") as f:
        while running:
            with np.load(embedding_path / f"{pos}.npz") as data:
                embedding_data = data["embs"]
            for embedding in embedding_data:
                if count >= max_size:
                    running = False
                    break
                if distance == "dot":
                    embedding = embedding / np.linalg.norm(embedding)
                query = create_query(embedding.tolist(), page_size, pre_filter)
                # print(query)
                took, hits, ids, scores = search(query)
                # print(f"{took}, {ids}, {scores}")
                if took == -1:
                    error_count += 1
                    if error_count >= max_error_count:
                        running = False
                        break
                    continue
                result = {
                    "id": (count + 1),
                    "took": took,
                    "hits": hits,
                    "ids": ids,
                    "scores": scores,
                }
                f.write(json.dumps(result, ensure_ascii=False))
                f.write("\n")
                count += 1
                if count % 10000 == 0:
                    print(f"Sent {count}/{max_size} queries.")

            pos += 100000
            if pos > num_of_docs:
                pos = 0

    execution_time = time.time() - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Execution Time: {int(hours):02d}:{int(minutes):02d}:{seconds:02.2f}")


In [ ]:
def get_output_filename(weaviate_version, name, explain=False, track_total_hits=False):
    filename = f"output/weaviate{weaviate_version.replace('.', '_')}_{name}"
    filename += ".jsonl.gz"
    return filename


In [ ]:
def print_took_and_total_hits(filename):
    tooks = []
    hits = []
    with gzip.open(filename, "rt", encoding="utf-8") as f:
        for line in f.readlines():
            obj = json.loads(line)
            tooks.append(obj.get("took"))
            hits.append(obj.get("hits"))
    df = pd.DataFrame({"took": tooks, "hits": hits})
    print(df.describe().to_markdown())
    return {
        "num_of_queries": len(df),
        "took": {
            "mean": df.took.mean(),
            "std": df.took.std(),
            "min": df.took.min(),
            "25%": df.took.quantile(0.25),
            "50%": df.took.quantile(0.5),
            "75%": df.took.quantile(0.75),
            "90%": df.took.quantile(0.9),
            "99%": df.took.quantile(0.99),
            "max": df.took.max(),
        },
        "hits": {
            "mean": df.took.mean(),
            "std": df.took.std(),
            "min": df.took.min(),
            "25%": df.took.quantile(0.25),
            "50%": df.took.quantile(0.5),
            "75%": df.took.quantile(0.75),
            "max": df.took.max(),
        },
    }


In [ ]:
def save_results():
    with open("results.json", "wt", encoding="utf-8") as f:
        json.dump({
            "version": weaviate_version,
            "settings": dataset_config,
            "results": results,
        }, f, ensure_ascii=False, default=lambda x: int(x) if isinstance(x, np.int64) else None)


In [ ]:
prune_docker()
print(f"<<<Weaviate {weaviate_version}>>>")
run_weaviate()
wait_for_weaviate()

In [ ]:
print_docker_container_stats()
print_docker_system_df()

In [ ]:
create_index()

In [ ]:
print_docker_container_stats()
print_indices()
print_docker_system_df()

In [ ]:
results["indexing"] = insert_data(bulk_size=bulk_size, max_size=index_size)

In [ ]:
results["indexing"]["container"] = print_docker_container_stats()
print_indices()
print_docker_system_df()

In [ ]:
for page_size in [10, 100, 400]:
    print(f"page size: {page_size}")
    filename = get_output_filename(weaviate_version, f"knn_{page_size}")
    search_with_knn_queries(filename, page_size=page_size, max_size=1000) # warmup
    search_with_knn_queries(filename, page_size=page_size, offset=index_size)
    results[f"top_{page_size}"] = print_took_and_total_hits(filename)

In [ ]:
def pre_filter_generator():
    if len(section_values) > 0:
        while True:
            for s in section_values:
                yield f"""
      where: {{
        operator: Equal
        path: ["section"]
        valueString: "{s}"
      }}
"""

for page_size in [10, 100, 400]:
    print(f"page size: {page_size}")
    filename = get_output_filename(weaviate_version, f"knn_{page_size}_filtered", explain=False, track_total_hits=False)
    search_with_knn_queries(filename, page_size=page_size, max_size=1000, pre_filter=pre_filter_generator()) # warmup
    search_with_knn_queries(filename, page_size=page_size, offset=index_size, pre_filter=pre_filter_generator())
    results[f"top_{page_size}_filtered"] = print_took_and_total_hits(filename)

In [ ]:
print_docker_container_stats()
print_indices()
print_docker_system_df()

In [ ]:
save_results()

In [ ]:
delete_index()
stop_weaviate()